In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/

!kaggle config set -n path -v /content 

!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d yingwurenjian/chicago-divvy-bicycle-sharing-data

!mkdir /content/divvy
!mkdir /content/divvy/data
!cp -r /content/datasets/yingwurenjian/chicago-divvy-bicycle-sharing-data /content/divvy/data

!ls /content/divvy/data

!unzip \
/content/divvy/data/chicago-divvy-bicycle-sharing-data/chicago-divvy-bicycle-sharing-data.zip \
-d /content/divvy/data/

!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
  
!pip install pyproj==1.9.6

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes
- path is now set to: /content
 98% 616M/629M [00:14<00:00, 35.6MB/s]
100% 629M/629M [00:14<00:00, 45.4MB/s]
chicago-divvy-bicycle-sharing-data
Archive:  /content/divvy/data/chicago-divvy-bicycle-sharing-data/chicago-divvy-bicycle-sharing-data.zip
  inflating: /content/divvy/data/data_raw.csv  
  inflating: /content/divvy/data/data.csv  
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libgeos-3.5.0
E: Couldn't find any package by glob 'libgeos-3.5.0'
E: Couldn't find any package by regex 'libgeos-3.5.0'
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
Suggested packages:
  libgdal-doc
The following NEW packages will be installed:
  libgeos-dev
0 

     |████████████████████████████████| 2.8MB 2.7MB/s 
  Created wheel for pyproj: filename=pyproj-1.9.6-cp36-cp36m-linux_x86_64.whl size=3702070 sha256=4aea143396daeea73b3cb4814258c82019dbe69854a12ad8c9125bda791eac2e
  Stored in directory: /root/.cache/pip/wheels/02/cd/b1/a2d6430f74c7a778a43d62f78bec109ca69c732dc9b929142a
Successfully built pyproj
  Found existing installation: pyproj 2.2.1
    Uninstalling pyproj-2.2.1:
      Successfully uninstalled pyproj-2.2.1


In [2]:
!wget -O /content/divvy/data/chicago_shapefiles.zip 'http://download.geofabrik.de/north-america/us/illinois-latest-free.shp.zip'
!unzip /content/divvy/data/chicago_shapefiles.zip -d /content/divvy/data/shapefiles/ && rm /content/divvy/data/chicago_shapefiles.zip
!ls /content/divvy/data/
!ls /content/divvy/data/shapefiles/

--2019-08-15 15:28:18--  http://download.geofabrik.de/north-america/us/illinois-latest-free.shp.zip
Resolving download.geofabrik.de (download.geofabrik.de)... 88.99.142.44, 88.99.105.98
Connecting to download.geofabrik.de (download.geofabrik.de)|88.99.142.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 319792287 (305M) [application/zip]
Saving to: ‘/content/divvy/data/chicago_shapefiles.zip’

/content/divvy/data 100%[===================>] 304.98M  10.8MB/s    in 30s     

2019-08-15 15:28:48 (10.2 MB/s) - ‘/content/divvy/data/chicago_shapefiles.zip’ saved [319792287/319792287]

Archive:  /content/divvy/data/chicago_shapefiles.zip
  inflating: /content/divvy/data/shapefiles/README  
 extracting: /content/divvy/data/shapefiles/gis_osm_buildings_a_free_1.cpg  
  inflating: /content/divvy/data/shapefiles/gis_osm_buildings_a_free_1.dbf  
  inflating: /content/divvy/data/shapefiles/gis_osm_buildings_a_free_1.prj  
  inflating: /content/divvy/data/shapefiles/gis_os

In [0]:
import os
import sys
import time
import random

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from mpl_toolkits.basemap import Basemap

In [2]:
csv_filename = '/content/divvy/data/data.csv'
data_df = pd.read_csv(csv_filename)
data_df.head()

,trip_id,year,month,week,day,hour,usertype,gender,starttime,stoptime,tripduration,temperature,events,from_station_id,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_id,to_station_name,latitude_end,longitude_end,dpcapacity_end
0,2355134,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:57:00,2014-07-01 00:07:00,10.066667,68.0,tstorms,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,303,Broadway & Cornelia Ave,41.945512,-87.645980,15.0
1,2355133,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:56:00,2014-07-01 00:00:00,4.383333,68.0,tstorms,282,Halsted St & Maxwell St,41.864580,-87.646930,15.0,22,May St & Taylor St,41.869482,-87.655486,15.0
2,2355130,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:33:00,2014-06-30 23:35:00,2.100000,68.0,tstorms,327,Sheffield Ave & Webster Ave,41.921687,-87.653714,19.0,225,Halsted St & Dickens Ave,41.919936,-87.648830,15.0
3,2355129,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:26:00,2014-07-01 00:24:00,58.016667,68.0,tstorms,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0,194,State St & Wacker Dr,41.887155,-87.627750,11.0
4,2355128,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:16:00,2014-06-30 23:26:00,10.633333,68.0,tstorms,320,Loomis St & Lexington St,41.872187,-87.661501,15.0,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0


In [0]:
ll_lon = -87.00
ll_lat = 41.00

ur_lon = -85.00
ur_lat = 42.00

plt.figure(figsize=(10,10))

# Generate the basemap
chicago_map = Basemap(
    
    projection='tmerc',
    
    llcrnrlon=-75,
    llcrnrlat=0,
    urcrnrlon=-25,
    urcrnrlat=50,
    
    lon_0=0,
    lat_0=0,
             
    resolution='c'
)

chicago_map.drawmapboundary(fill_color='aqua')
# chicago_map.fillcontinents(color='#ddaa66',lake_color='aqua')
chicago_map.drawstates()
chicago_map.drawcoastlines()

shapefiles = os.listdir('/content/divvy/data/shapefiles/')
road_shapefiles = [file for file in shapefiles if 'roads' in file]
chicago_shapefile_string = road_shapefiles[0][:-4]

data_path = '/content/divvy/data/'
shapefile_full_path = data_path + 'shapefiles/' + chicago_shapefile_string
print(shapefile_full_path)

chicago_map.readshapefile(shapefile_full_path, 'chicago_streets')

plt.show()